## 形態素解析ツールについて

### MeCab
[MeCab](http://taku910.github.io/mecab/)は長らく日本語の形態素解析ツールとして使われてきたが,

1. 2013年以降更新されていない
1. ややbuildが複雑
1. ネット上の記事において, 使用する辞書に関する記述が古い

などの問題がある. 形態素解析の質は辞書によるところも大きいので, １つ目の問題はあまり問題にはなっていない.
２つめの問題もPythonから呼び出す限りにおいては,
[mecab-python3](https://github.com/SamuraiT/mecab-python3)
をpipで導入すれば使えるため, 比較的小さい問題である （コマンドラインからは利用できないが.）

辞書に関しては, IPAdicは2007年に, NEologdは2020年から更新されていないので,
[mecab-python3](https://github.com/SamuraiT/mecab-python3)
で推奨されている[UniDic](https://clrd.ninjal.ac.jp/unidic/)を選ぶのがよいだろう.
軽量版の`unidic-lite`はpipから, 完全版は
```
pip install unidic
python3 -m unidic download
```
でダウンロードすることができる.

### Sudachi
[Sudachi](https://github.com/WorksApplications/Sudachi)はワークスアプリケーションズ
の徳島人工知能NLP研究所が開発している形態素解析ツール.
リクルートのMegagon Labsと国立国語研究所が開発している
日本語NLPライブラリの[GiNZA](https://spacy.io/)の中で, 形態素解析器として用いられている.
GiNZAは欧米諸言語で広く利用されている[spaCy](https://spacy.io/)の
日本語対応版といったライブラリでこれから広く利用されると見込まれている.

Sudachiの辞書はUniDicとNEologdをベースに独自の修正や語彙追加が施されている.
詳細については, 開発者の一人の久本空海さんによる
[解説](https://zenn.dev/sorami/articles/c9a506000fd1fbd1cf98)
などを参考にされたい.


## 3.1
>3.1節にあるように, 実際に形態素解析を行ってみよう.
>3.1節と同じ結果が出ただろうか.
>MeCabを使った形態素解析においては, どの辞書を使うかによって解析結果が異なってくる.
>MeCabの辞書としてどのようなものがあるか調べてみよう.
>また辞書を変えたとき, 解析結果がどのように変わるか調べてみよう.

### MeCab


In [1]:
import MeCab
import unidic_lite
mecab_lite = MeCab.Tagger(f'-d "{unidic_lite.DICDIR}"')
#import unidic
#mecab_full = MeCab.Tagger(f'-d "{unidic.DICDIR}"')

In [2]:
#| code-fold: false
print(mecab_lite.parse("日本語の文の形態素解析を行った."))

日本	ニッポン	ニッポン	日本	名詞-固有名詞-地名-国			3
語	ゴ	ゴ	語	名詞-普通名詞-一般			1
の	ノ	ノ	の	助詞-格助詞			
文	ブン	ブン	文	名詞-普通名詞-一般			1
の	ノ	ノ	の	助詞-格助詞			
形態	ケータイ	ケイタイ	形態	名詞-普通名詞-一般			0
素	ソ	ソ	素	接尾辞-名詞的-一般			
解析	カイセキ	カイセキ	解析	名詞-普通名詞-サ変可能			0
を	オ	ヲ	を	助詞-格助詞			
行っ	オコナッ	オコナウ	行う	動詞-一般	五段-ワア行	連用形-促音便	0
た	タ	タ	た	助動詞	助動詞-タ	終止形-一般	
.			．	補助記号-句点			
EOS



In [3]:
#| code-fold: false
#print(mecab_full.parse("日本語の文の形態素解析を行った."))

### Sudachi (GiNZA)


In [4]:
import spacy
nlp = spacy.load("ja_ginza")

doc = nlp("日本語の文の形態素解析を行った.")

for sent in doc.sents:
    for token in sent:
        print(token.i, token.orth_, token.lemma_, token.pos_, 
              token.tag_, token.dep_, token.head.i)

0 日本語 日本語 NOUN 名詞-普通名詞-一般 nmod 2
1 の の ADP 助詞-格助詞 case 0
2 文 文 NOUN 名詞-普通名詞-一般 nmod 5
3 の の ADP 助詞-格助詞 case 2
4 形態素 形態素 NOUN 名詞-普通名詞-一般 compound 5
5 解析 解析 NOUN 名詞-普通名詞-サ変可能 obj 7
6 を を ADP 助詞-格助詞 case 5
7 行っ 行う VERB 動詞-一般 ROOT 7
8 た た AUX 助動詞 aux 7
9 . . PUNCT 補助記号-句点 punct 7


現在のサポート状況やライブラリの数などを鑑みて, 以降はGiNZAを用いる.

## 3.2
>新聞記事など, 身近なテキストをダウンロードし, それを形態素解析してみよう.
>その上で, 出現する形態素ごとにその形態素が出現する頻度を数えるプログラムを書いてみよう.
>その結果は元のテキストの特徴を表しているだろうか？


In [5]:
import pandas as pd

url = "https://nlp100.github.io/data/neko.txt"
df = pd.read_csv(url, header = None)
df.rename(columns={0 : "raw"}, inplace=True)

indices = [df.index[df.raw == s][0] for s in ["一", "二", "三", "四", "五", "六", "七", "八", "九", "十", "十一"]]
indices.append(len(df))
chapter = []
for i in range(1, len(indices)):
    chapter.extend([i] * (indices[i] - indices[i-1]))

df["chapter"] = chapter
df.drop(index = indices[0:-1], inplace = True)
df.reset_index(drop=True, inplace = True)

In [6]:
docs = [nlp(raw) for (raw, chap) in zip(df.raw, df.chapter) if chap == 1]

In [7]:
from collections import defaultdict

d_count = defaultdict(int)
d_count_noun = defaultdict(int)
for doc in docs:
    for sent in doc.sents:
       for token in sent:
            if token.pos_ != "PUNC":
                d_count[token.lemma_] += 1
                if token.pos_ == "NOUN":
                    d_count_noun[token.lemma_] += 1

d_count = sorted(d_count.items(), key=lambda x: x[1], reverse=True)
d_count_noun = sorted(d_count_noun.items(), key=lambda x: x[1], reverse=True)

:::: {.columns}
::: {.column}

**すべての形態素**


In [8]:
d_count[0:10]

[('の', 384),
 ('。', 329),
 ('て', 308),
 ('だ', 289),
 ('は', 271),
 ('を', 252),
 ('と', 239),
 ('た', 218),
 ('に', 217),
 ('が', 207)]

:::
::: {.column}

**名詞のみ**


In [9]:
d_count_noun[0:10]

[('吾輩', 81),
 ('もの', 53),
 ('事', 49),
 ('主人', 38),
 ('猫', 23),
 ('人', 19),
 ('人間', 18),
 ('時', 18),
 ('眼', 18),
 ('家', 18)]

:::
::::

## 3.3
>口語的な文では係り受けに交差が生じることがある.
>係り受けに交差が生じている文を考えてみよう.
>またそのような文を解析するためにはどのような考え方が必要になるか, 考えてみよう.

「これが私は正しいと思う.」

- これ $\rightarrow$ 正しい
- 私 $\rightarrow$ 思う


In [10]:
import ginza
text = "これが私は正しいと思う."
doc = nlp(text)

for span in ginza.bunsetu_phrase_spans(doc):
    for token in span.lefts:
        print(f'{token} : {str(ginza.bunsetu_span(token))} → {str(span)}')

これ : これが → 正しい
私 : 私は → 正しい
正しい : 正しいと → 思う


係り受けに交差を許す場合, すべての2単語のペアについて解析する必要がある.
そのため非効率 $(O(n^3))$ に思われていたが, @mcdonald_non-projective_2005
はこの Non-projective dependency parsing 問題が, 最小全域木問題に帰着し, $O(n^2)$
の計算量であることを示した.


## 3.4
>新聞記事などから一つ文を選び, その分の修辞構造木を作成してみよう.
>作成したら, 友人などにも同じように修辞構造木を作成してもらい, 自分のものと比較してみよう.
>修辞構造木は注釈付が難しいことが知られており,
>同じ文に対しても注釈者によって別の構造が付与されることがままある.
>その原因についても考えてみよう.

Skip.